# 7 - Multi-Agent
## Iterative Workflows - Loop Agent
- This code snippet demonstrates the use of the LoopAgent in Google ADK to create iterative workflows. 
- The LoopAgent is designed to execute its configured sub_agents sequentially within a loop. A custom BaseAgent called CheckCondition is defined to inspect the InvocationContext's session state and determine if a termination condition is met. The loop continues running the process_step (an LlmAgent) and the Checker repeatedly. 
- The iteration halts either when the CheckCondition agent yields an Event with escalate=True or when the specified max_iterations is reached. 
- The consistent passing of the same InvocationContext across iterations allows agents within the loop to share and modify state, enabling complex iterative logic.


In [1]:
# Conceptual Example: Loop with Condition
from google.adk.agents import LoopAgent, LlmAgent, BaseAgent
from google.adk.events import Event, EventActions
from google.adk.agents.invocation_context import InvocationContext
from typing import AsyncGenerator

class CheckCondition(BaseAgent): # Custom agent to check state
    async def _run_async_impl(self, ctx: InvocationContext) -> AsyncGenerator[Event, None]:
        status = ctx.session.state.get("status", "pending")
        is_done = (status == "completed")
        yield Event(author=self.name, actions=EventActions(escalate=is_done)) # Escalate if done

process_step = LlmAgent(name="ProcessingStep") # Agent that might update state['status']

poller = LoopAgent(
    name="StatusPoller",
    max_iterations=10,
    sub_agents=[process_step, CheckCondition(name="Checker")]
)
# When poller runs, it executes process_step then Checker repeatedly
# until Checker escalates (state['status'] == 'completed') or 10 iterations